**Import Necessari**

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.callbacks import Callback
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

**Funzioni Utilizzate**

In [ ]:
def generator(dataset,batchsize,moves=10):
  while True:
    rand = np.random.randint(dataset.shape[0],size=batchsize)
    y_truth = dataset[rand]
    blurred = np.copy(y_truth)
    moving = tf.expand_dims(y_truth,axis=-1)

    for i in range(moves):
      #RandomTranslation requires a channel axis
      moving = layers.RandomTranslation(0.07,0.07,fill_mode='constant',interpolation='bilinear')(moving)
      blurred = blurred + tf.squeeze(moving)
    blurred = blurred/(moves+1)
    yield(blurred,y_truth)

def plot(x,y):
  no = x.shape[0]
  print(no)
  plt.figure(figsize=(2*no, 4))
  for i in range(no):
    # display original
    ax = plt.subplot(2, no, i + 1)
    plt.imshow(x[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, no, i + 1 + no)
    plt.imshow(y[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
  plt.show()

def deblur_model(input_shape):
  inputs = layers.Input(shape=input_shape)

  # Encoder
  x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
  x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)

  # Decoder
  x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  outputs = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

  model = Model(inputs, outputs)
  return model

**Definizione del Modello**

In [ ]:
(x_train, y_train), (x_val,y_val) = mnist.load_data()
x_train = (x_train/255.).astype(np.float32)
x_val = (x_val/255.).astype(np.float32)

#cambiare rapporto di grandezza tra train-val-test set

**Fase di Training del Modello**

In [2]:
#parametri
batch_size = 32
epochs = 10
new_learning_rate = 0.001
new_optimizer = Adam(learning_rate=new_learning_rate)
steps_per_epoch=len(x_train) // batch_size

#dati di training
my_gen = generator(x_train,batch_size)
train_x,train_y = next(my_gen)
#plot(train_x,train_y)

#dati di validazione
my_gen2 = generator(x_val,batch_size)
val_x,val_y = next(my_gen2)

model = deblur_model((28, 28, 1))
model.compile(optimizer=new_optimizer, loss='mse', metrics=['accuracy'])

#model.load_weights("mnist_project.h5")

history = model.fit(
    my_gen,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    validation_data=(val_x,val_y),
    shuffle=True
)

mse_values_valid = history.history['val_loss']

mean_mse_valid = np.mean(mse_values_valid)
std_mse_valid = np.std(mse_values_valid)

print(f'\n Mean MSE (Validation): {mean_mse_valid}')
print(f'Standard Deviation MSE (Validation): {std_mse_valid}')

model.summary()

model.save_weights("mnist_project.h5")

#utilizzare history restituito dal fit per fare un plotting delle statistiche che abbiamo su accuracy e loss

# batch size : 32
# epochs = 10
# learning rate non specificato
# val_loss: 0.0098 - val_accuracy: 0.8111

# Mean MSE (Validation): 0.017578958347439767
# Standard Deviation MSE (Validation): 0.005811503020716015
# con parametri: batch 32, epochs 10, lr 0.001

NameError: name 'generator' is not defined

**Fase di Validazione del Modello**

In [ ]:
#da consegna credo che val_x e val_y (validation set) debbano essere 10k immagini
(loss_value, acc_value) = model.evaluate(my_gen2, steps=100) #len(x_val) // batch_size

deblurred_images = model.predict(val_x, batch_size)

deblurred_images_squeezed = np.squeeze(deblurred_images)

#mse values
mse_values_valid = np.mean(np.square(deblurred_images_squeezed - val_y), axis=1)

# Calcola la deviazione standard dell'MSE
std_mse_valid = np.std(mse_values_valid)

print(f"\n MSE: {loss_value}")
print(f'\n Standard Deviation MSE: {std_mse_valid}')
print(f"\n Accuracy: {acc_value}")

#mean_square_error(test_x,valid_x);

**Plotting**

In [ ]:
plot(val_x, deblurred_images)